In [ ]:
%matplotlib inline
%run ../setup/nb_setup

# Science Case Studies: The Age-Velocity Dispersion Relations in GALAH DR3

Author(s): Keith Hawkins


## Learning goals

The purpose of this tutorial is to: 
1. Introduce the concepts of Age-Velocity Dispersion relations
2. Introduce Cross matching via topcat
3. Introduce/practice velocity calculations via astropy 

Goal: You will derive the Gaia DR2-GALAH Age-Velocity Dispersion relations. Two additional challenges include: (1) derive the same Age-Velocity Dispersion relations but for  Gaia DR3-GALAH (hint!: this will require a cross match) and (2) consider uncertainties in age and velocity to build a linear model which converts velocity dispersion into age.


### Notebook Setup and Package Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as p
import astropy
from astropy.table import Table
import scipy.stats
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.coordinates import Galactic
from astropy.coordinates import ICRS
import astropy.coordinates as apycord
import random

We will need to download the GALAH DR3 dataset that contains age information. This dataset also contains DR2 estimates of velocity but *NOT* DR3 (You will need to do that cross match):

DATA LOCATION : https://cloud.datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/

Local mirror : https://users.flatironinstitute.org/~apricewhelan/data/surveys/GALAH/GALAH_DR3_VAC_ages_v2.fits

Main GALAH Table :

-GALAH_DR3_main_allstar_v2.fits -- Main GALAH DR3

-GALAH_DR3_VAC_ages_v2.fits -- GALAH DR3 ages

-GALAH_DR3_VAC_GaiaEDR3_v2.fits	-- GaiaEDR3 x GALAH


Download this dataset
and load it in with astropy!



In [ ]:
GALAH = Table.read("GALAH_master_age.fits")  # loading in the data

In [ ]:
# explore the columns:

print(GALAH.colnames)

The goal here is to determine how velocity (and velocity dispersion) is correlated with age. 

As such we will need to find the (precomputed DR2) velocity columns; and age columns (based on the cell above). 

Not all velociities and ages are measured with the same precision so we will also want to apply some very simple quality control cuts. So lets do that in the next cell

In [ ]:
# lets create a quality
v_err_lim = 5  # define the limiting velocity uncertinaty in all axes
age_err_lim = 0.3  # age uncertainty limit in percent
dist_err_lim = 0.15  # distance error in percent
GALAH = GALAH[
    GALAH["parallax_edr3"] > 0
]  # Makes sure no distances will be zero for astropy later

# Quality cuts
ok = np.where(
    (GALAH["vR_Rzphi_50"] - GALAH["vR_Rzphi_5"] < v_err_lim)
    & (GALAH["vT_Rzphi_50"] - GALAH["vT_Rzphi_5"] < v_err_lim)
    & (GALAH["vz_Rzphi_50"] - GALAH["vz_Rzphi_5"] < v_err_lim)
    & (GALAH["e_age_bstep"] / GALAH["age_bstep"] < age_err_lim)
    & (GALAH["parallax_error"] / GALAH["parallax"] < dist_err_lim)
    & (GALAH["age_bstep"] > 1)
    & (GALAH["age_bstep"] < 10)
)[0]

print("There are %i stars that pass the quality controls" % len(ok))
D = GALAH[ok]

With the 'quality' sample in hand, lets now try to figure out how velocity and velocity dispersion depends on age in this sample. We will start by simply plotting velocity in all directions as a function of age.

**EXERCISE: PLOT how each velocity component depends on age!**

In [ ]:
# Now we want to plot how the velocities depends on age


From the above plots we can see that:
1. Mostly velocities are indepdent of age (no signficant relationships), except maybe in V_theta
2. The velocity dispersion in all velcoity directons likley grows with increasing age

So lets bin by age and see if we can determine the relationship between age and velocity.

In [ ]:
# for each star we will place them into bins by age using scipy.stats.binned_statistic

age_bins = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # define the limits of each bin

## With the ages now binned up lets plot compute the bin's mean age and dispersion in age for each velocity direction
mean_VR, bin_edge, inds = scipy.stats.binned_statistic(
    D["age_bstep"], D["vR_Rzphi_50"], statistic="mean", bins=age_bins
)
std_VR, bin_edge, inds = scipy.stats.binned_statistic(
    D["age_bstep"], D["vR_Rzphi_50"], statistic="std", bins=age_bins
)

mean_Vz, bin_edge, inds = scipy.stats.binned_statistic(
    D["age_bstep"], D["vz_Rzphi_50"], statistic="mean", bins=age_bins
)
std_Vz, bin_edge, inds = scipy.stats.binned_statistic(
    D["age_bstep"], D["vz_Rzphi_50"], statistic="std", bins=age_bins
)

mean_Vt, bin_edge, inds = scipy.stats.binned_statistic(
    D["age_bstep"], D["vT_Rzphi_50"], statistic="mean", bins=age_bins
)
std_Vt, bin_edge, inds = scipy.stats.binned_statistic(
    D["age_bstep"], D["vT_Rzphi_50"], statistic="std", bins=age_bins
)

bincen = [
    (bin_edge[i + 1] + bin_edge[i]) / 2.0 for i in range(len(age_bins) - 1)
]  # this set the bin centers

Now that we have binned the data and determined the mean and dispersion in each velocity direction, as a sanity check (which are incredibly important to do), lets overplot the data and the binned results to ensure it looks ok.

# The Age-Velocity Dispersion Relations in GALAH-Gaia DR2


Now we are ready to plot the age velocity disperison relations for each component for GALAH-Gaia DR2.

**EXERCISE : Plot how each velocity component is depended on age and also overplot the binned data from the previous cell** This will enable us to see how age and velocity (and velocity disperson) are related.

With the data binned we can now explore the velocity disperison as a function of age

**EXERCISE: now plot the velocity dipserison as a function of age**

*We now see that the age-velocity dispersion relations are roughly linear such that VELOCITY DISPERSION INCREASES with INCREASING age as expect*

# The Age-Velocity Dispersion Relations in GALAH-Gaia EDR3!

Let us now redo the above but with EDR3 data instead! This will require us to compute the velocities (and their uncertainties -- challenge) from the EDR3 data directly.

You will need to first cross-match (via ADQL/TOPCAT/TAP QUERY/ASTROQUERY) orginal table with ED3. We will explore how to use topcat in this workshop but its trivial to do this via an ADQL/TAP Query.

Once the cross match is done, lets read it in.
Also lets build a function to compute the velocities from the observables 

**EXCERICSE: PLEASE build a function to compute the velocities from the uncertainties)**

In [ ]:
# --- ok lets now compute some velocities with EDR3 data! We start by making a defintion. MAKE SURE TO GO THROUGH THIS 
def compute_vels(ra, dec, pmra, pmdec, rv, dist, V0=[11.1, 245.0, 7.25], R0=8.3):


With the definition, we can now convert the observables into velocities by calling the defintion from above. We can also do this iterativley where in each iteration we perturb the input observables by the uncertainties. This will enable us to estimate the velocity uncertainties via monte carlo realizations.

**CHALLENGE EXERCISE: Try to compute the velocities and their uncertainties using  monte carlo realizations. Once computed add these to the astropy table as galvr_dr3, galvt_dr3, galvz_dr3,galvr_err_dr3,galvt_err_dr3,galvz_err_dr3**

Now that we have the velocities and their uncertainties with EDR3 data, lets make the same quality control cuts as before.

In [ ]:
ok = np.where(
    (GALAH["galvr_err_dr3"] < v_err_lim)
    & (GALAH["galvt_err_dr3"] < v_err_lim)
    & (GALAH["galvz_err_dr3"] < v_err_lim)
    & (GALAH["e_age_bstep"] / GALAH["age_bstep"] < age_err_lim)
    & (GALAH["parallax_error"] / GALAH["parallax"] < dist_err_lim)
    & (GALAH["age_bstep"] > 1)
    & (GALAH["age_bstep"] < 10)
)[0]

print("There are %i stars that pass the quality controls" % len(ok))
D_DR3 = GALAH[ok]

We can now plot the EDR3 version of the age velocity relations.

In [ ]:
p.figure(figsize=(10, 10))
ax1 = p.subplot(3, 1, 1)
p.plot(D_DR3["age_bstep"], D_DR3["galvr_dr3"], "k.")
# p.errorbar(bincen,mean_VR,yerr=std_VR, fmt='s',color='r',zorder=3 )
p.ylabel("V$_R$ (km/s)")

ax2 = p.subplot(3, 1, 2, sharex=ax1)
p.plot(D_DR3["age_bstep"], D_DR3["galvt_dr3"], "k.")
# p.errorbar(bincen,mean_Vt,yerr=std_Vt, fmt='s',color='r',zorder=3 )
p.ylabel(r"V$_{\theta}$ (km/s)")

ax3 = p.subplot(3, 1, 3, sharey=ax1)
p.plot(D_DR3["age_bstep"], D_DR3["galvz_dr3"], "k.")
# p.errorbar(bincen,mean_Vz,yerr=std_Vz, fmt='s',color='r',zorder=3 )
p.ylabel("V$_z$ (km/s)")
p.xlabel("age (Gyr)")
p.tight_layout()

We must now bin the data in the same way (by age) as for the DR3 results and compute the mean and std of the velocity in each age bin.

In [ ]:
mean_VR_dr3, bin_edge, inds = scipy.stats.binned_statistic(
    D_DR3["age_bstep"], D_DR3["galvr_dr3"], statistic="mean", bins=age_bins
)
std_VR_dr3, bin_edge, inds = scipy.stats.binned_statistic(
    D_DR3["age_bstep"], D_DR3["galvr_dr3"], statistic="std", bins=age_bins
)

mean_Vz_dr3, bin_edge, inds = scipy.stats.binned_statistic(
    D_DR3["age_bstep"], D_DR3["galvz_dr3"], statistic="mean", bins=age_bins
)
std_Vz_dr3, bin_edge, inds = scipy.stats.binned_statistic(
    D_DR3["age_bstep"], D_DR3["galvz_dr3"], statistic="std", bins=age_bins
)

mean_Vt_dr3, bin_edge, inds = scipy.stats.binned_statistic(
    D_DR3["age_bstep"], D_DR3["galvt_dr3"], statistic="mean", bins=age_bins
)
std_Vt_dr3, bin_edge, inds = scipy.stats.binned_statistic(
    D_DR3["age_bstep"], D_DR3["galvt_dr3"], statistic="std", bins=age_bins
)

bincen = [
    (bin_edge[i + 1] + bin_edge[i]) / 2.0 for i in range(len(age_bins) - 1)
]  # this set the bin centers

Finally, we plot the age velocity dispersion relations for GALAH-DR2 and GALAH-EDR3 datasets. 

**EXERCISE: Now plot the age-velocity dispersion relations (using the binned data from above) for EDR3 and DR2**

*We now see that the age-velocity dispersion relations are roughly linear such that VELOCITY DISPERSION INCREASES with INCREASING age as expected. If there are some differences between DR2 and EDR3, why might they exist.*

# Reach Challenge I: Fitting the Age-Velocity & Age-Velocity Dispersion Relations Simultaneously with Linear Model and MCMC 

Challenge: If you finish, try to figure out a way to define the age velocity relation *without* binning the data. (Hint: This can be done via writing down a linear model for how velocity dispersion depends on age and then comparing that model in the data space.)

In [ ]:
# so lets begin this by importing the required packages to fit the age velocity relation 
import emcee
import corner
import scipy
import scipy.stats

So we need to write down a model 

So we will model the velocity distribution as a function of age such that

$$V_i = m_{V_i-age} * age + b_{V_i-age}$$

where i represents the velocity component (i.e., i = {r, T, z} ), $m_{V_i-age}$ (called m in code) represents the slope of the age-velocity relation, and $b_{V_i-age}$ (called b2 in the code) represents the intercept of the age-velocity relation

Additionally, we will mdoel the velocity dispersion distribtion as a function of age such that:

$$\sigma V_i = m_{\sigma V_i-age} * age + b_{\sigma V_i-age}$$

Here, $m_{\sigma V_i-age}$ (called m in the code) represents the slope of the age-velocity disperison relation, and $b_{\sigma V_i-age}$ (called b in the code) represents the intercept of the age-velocity dispersion relation

Now that we have written down a linear model, which describes how both velocity and velocity disperison depends on age, we will need to fit the model to the data. Remember that the data we have is the measured velocities ($V_i$).

We will do this in a Bayesian way.

So lets quickly recap. 

Bayes theorm states that the probability of our model given the data that we have observed (i.e., $p(\theta∣data)$, the postieror) is proportional to the probility of measuring the data that we do given the model ($p(data∣\theta)$ often called the liklihood function) multiplied by the prior information we have that our model parameters are correct ($p(\theta)$, the prior). This is mathematically,

$$p(\theta∣data) \propto  p(data | \theta) * p(\theta) $$

In our specific case, $\theta$ represents our model parameters. Thus  $\theta$ = { $m_{V_i-age}$, $b_{V_i-age}$, $m_{\sigma V_i-age}$, $b_{\sigma V_i-age}$ }.

ultimately, we need to fit these model parameters by maximizing the posterior of the model. From above we can see that we can do this by maximizing the product between the liklihood and the prior. 

With this info in hand, lets write down our priors and likihood function.

$$ p(data | \theta) = \mathcal{N}(m_{V_i-age}*age+ b_{V_i-age}, sqrt(m_{\sigma V_i-age}*age+ b_{\sigma V_i-age} + \sigma_{V_i}) $$

and we set priors such that $m_{\sigma V_i-age}$, $b_{\sigma V_i-age}$ > 0 (i.e., the slope and intercept of the age velocity dispersion relations must be posititve to ensure that the velocity dispersion is in fact larger than 0. 

Now that we have all the background and info we need, lets go ahead and begin to write down our model, log likihood function, prior and emcee instance to fit the model via MCMC



In [ ]:
#Use this cell to write down the log liklihood, the log prior, and the emcee instance. 
#Make sure you use age, Vz and its uncertaintiy as the 'data' (if you get stuck use the soultions as a guide)


In [ ]:
#--lets flatten the samples and make the corner plot to see our results.
flat_samples = sampler.get_chain(discard=500, flat=True)
p.plot(flat_samples[:,0]) #in case you want to see one of the chains to make sure its converged.
labels = [r'm$_{\sigma V_z-age}$',r'b$_{\sigma V_z-age}$','m$_{V_z-age}$','b$_{V_z-age}$']
fig = corner.corner(flat_samples, labels=labels,)

In [ ]:
print('Now we see that the slope of the age-velocity dispersion (Z) relation is %.2f +/- \
%.2f km/s per Gyr FOR GALAH'%(np.median(flat_samples[:,0]),np.std(flat_samples[:,0])))


print('Finally, we find that the slope of the age-velocity (Z) relation is %.2f +/- \
%.2f km/s per Gyr FOR GALAH'%(np.median(flat_samples[:,2]),np.std(flat_samples[:,2])))



## Challenge II: If we want to measure the age of an unknown population of stars, is it better to use the Age-velocity dispersion (in Z) relation or the Age-velocity relation (in theta)?

To answer this question we must determine whether the slope of the Age-velocity relation in theta is steeper (or not) compared to the age-velocity dispersion relation in Z. Whichever has the steeper slope changes more drastically with age and is thus a 'better' relation to use. So lets refit our above model for VT instead of Vz

In [ ]:
#Use this cell to re-initalize the emcee model but this time use Vtheta not Vz

In [ ]:
#--lets flatten the samples and make the corner plot to see our results.
flat_samples = sampler.get_chain(discard=500, flat=True)
p.plot(flat_samples[:,0]) #in case you want to see one of the chains to make sure its converged.
labels = [r'm$_{\sigma V_\theta-age}$',r'b$_{\sigma V_\theta-age}$',r'm$_{V_\theta-age}$',r'b$_{V_\theta-age}$']
fig = corner.corner(flat_samples, labels=labels,)

print('Now we see that the slope of the age-velocity dispersion (theta) relation is %.2f +/- \
%.2f km/s per Gyr FOR GALAH'%(np.median(flat_samples[:,0]),np.std(flat_samples[:,0])))


print('Finally, we find that the slope of the age-velocity (theta) relation is %.2f +/- \
%.2f km/s per Gyr FOR GALAH'%(np.median(flat_samples[:,2]),np.std(flat_samples[:,2])))